# If US States were individual countries, how do they compare?

## 1. Getting the data with API

In [ ]:
!pip install cenpy
import cenpy
import numpy as np
np.random.seed(42)

### **1.1 ACS 5-Year Estimate by US States**
1. Median Household Income
2. Education Levels
3. Unemployment rate
4. Labour force participation rate
5. Population
6. Poverty rate

In [ ]:
# Finding data set
available = cenpy.explorer.available()
available

# We use data from ACS 5 Year, Monthly Export and Import

In [ ]:
acs = available.filter(regex="^ACS", axis=0)
available.filter(regex="^ACSDT5Y", axis=0)

In [4]:
cenpy.explorer.explain("ACSDT5Y2023")
acs = cenpy.remote.APIConnection('ACSDT5Y2023')

In [ ]:
acs.varslike(
    pattern="B19019_001E",
    by="attributes",
).sort_index()

In [6]:
acs_variables = [
    "NAME",
    "GEO_ID",
    "B19019_001E", # Total Median Household Income
    "B06009_001E", # Educational Attainment- Total
    "B06009_002E", # Educational Attainment- Less than high school graduate
    "B06009_003E", # Educational Attainment- High school graduate (equivalent)
    "B06009_004E", # Educational Attainment- Some college or associate's degree
    "B06009_005E", # Educational Attainment- Bachelor's degree or higher
    "B01003_001E", # Total Population
    "B23025_001E", # Total Population above 16
    "B23025_002E", # Labor Force- Total
    "B23025_005E", # Labour Force- Unemployed
    "B17020_001E", # Population for whom poverty is determined
    "B17020_002E", # Population below poverty level
]

#1. Median Household Income [x]
#2. Education Levels [x]
#3. Unemployment rate [x]
#4. Labour force participation rate [x]
#5. Population [x]
#6. Poverty rate [x]



In [ ]:
# Filter out the data for state
USA_acs_data = acs.query(
    cols=acs_variables,
    geo_unit="state:*",
)

USA_acs_data.head()

In [8]:
for variable in acs_variables:
    # Convert all variables EXCEPT for NAME
    if variable not in ("NAME", "GEO_ID"):
        USA_acs_data[variable] = USA_acs_data[variable].astype(float)

In [9]:
USA_acs_data = USA_acs_data.rename(
    columns={
        "B19019_001E": "MedHHInc", # Total Median Household Income
        "B06009_001E": "EducTotal", # Educational Attainment- Total
        "B06009_002E": "EducBelowHighSch", # Educational Attainment- Less than high school graduate
        "B06009_003E": "EducHighSch", # Educational Attainment- High school graduate (equivalent)
        "B06009_004E": "EducAssoc", # Educational Attainment- Some college or associate's degree
        "B06009_005E": "EducBach", # Educational Attainment- Bachelor's degree or higher
        "B01003_001E": "TotalPop", # Total Population
        "B23025_001E": "TotalPop16", # Total Population above 16
        "B23025_002E": "LabForTotal", # Labor Force- Total
        "B23025_005E": "Unemployed", # Labour Force- Unemployed
        "B17020_001E": "PopPovertyDetermined", # Population for whom poverty is determined
        "B17020_002E": "PovertyPop", # Population below poverty level
    }
)

*Feature engineering*

In [10]:
USA_acs_data['PctBach'] = USA_acs_data['EducBach']/USA_acs_data['EducTotal']
USA_acs_data['PovertyRate'] = USA_acs_data['PovertyPop']/USA_acs_data['PopPovertyDetermined']
USA_acs_data['UnemploymentRate'] = USA_acs_data['Unemployed']/USA_acs_data['LabForTotal']
USA_acs_data['LabForParticipationRate'] = USA_acs_data['LabForTotal']/USA_acs_data['TotalPop16']

#Remove Puerto Rico
USA_acs_data = USA_acs_data[USA_acs_data['NAME'] != 'Puerto Rico']

### **1.2 International Trade**

*Exports*

In [11]:
cenpy.explorer.explain("ITMONTHLYEXPORTSSTATENAICS")
US_export = cenpy.remote.APIConnection('ITMONTHLYEXPORTSSTATENAICS')


In [ ]:
US_export.variables

In [ ]:
US_export.geographies['fips']

In [ ]:
US_export_data = US_export.query(
    cols=["US_STATE", "GEO_ID","YEAR","MONTH","ALL_VAL_YR"],
    geo_unit="world:*",
    )

for variable in ["YEAR","MONTH","ALL_VAL_YR"]:
    US_export_data[variable] = US_export_data[variable].astype(float)

US_export_data

In [15]:
# Filter data for December 2023
US_export_2023 = US_export_data[(US_export_data['YEAR'] == 2023) & (US_export_data['MONTH'] == 12)]

*Imports*

In [16]:
cenpy.explorer.explain("ITMONTHLYIMPORTSSTATENAICS")
US_import = cenpy.remote.APIConnection('ITMONTHLYIMPORTSSTATENAICS')

In [ ]:
US_import.variables

In [ ]:
US_import_data = US_import.query(
    cols=["US_STATE", "GEO_ID","YEAR","MONTH","GEN_VAL_YR"],
    geo_unit="world:*",
    )

for variable in ["YEAR","MONTH","GEN_VAL_YR"]:
    US_import_data[variable] = US_import_data[variable].astype(float)

US_import_data

In [19]:
# Filter data for December 2023
US_import_2023 = US_import_data[(US_export_data['YEAR'] == 2023) & (US_import_data['MONTH'] == 12)]

*Net Export (Export-Import)* 

In [20]:
# Join export and import data
US_netexport_2023 = US_export_2023.merge(US_import_2023[['US_STATE', 'GEN_VAL_YR']], on='US_STATE', how='left')

# Net export
US_netexport_2023["netexport"] = US_netexport_2023["ALL_VAL_YR"]-US_netexport_2023["GEN_VAL_YR"]

In [21]:
# Create new column with states name
state_names = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia'
}

US_netexport_2023['STATE_NAME'] = US_netexport_2023['US_STATE'].map(state_names)

# Filter states only
US_netexport_2023 = US_netexport_2023.dropna(subset=['STATE_NAME'])

### **1.3 Bureau of Economic Analysis: Gross Domesitc Product (GDP)**

In [ ]:
!pip install beaapi-0.0.2-py3-none-any.whl

import beaapi

In [23]:
beakey = '60AB5AC4-7ED2-481D-B559-BB6000F924ED'

In [ ]:
# List of data set available
list_of_sets = beaapi.get_data_set_list(beakey)
# List of parameters
list_of_params = beaapi.get_parameter_list(beakey, 'Regional')
# List of parameters values
list_of_param_vals = beaapi.get_parameter_values(beakey, 'Regional', 'LineCode',)
list_of_param_vals

In [ ]:
bea_tbl = beaapi.get_data(beakey, datasetname='Regional', GeoFips= 'STATE', LineCode= '1', TableName='SAGDP9N', Year='2023')
display(bea_tbl.head(5))

In [ ]:
bea_state_tbl = bea_tbl[bea_tbl['GeoName'].isin(state_names.values())]
bea_state_tbl.rename(columns={'DataValue': 'REALGDP'}, inplace=True)

### **1.4 Centers for Disease Control and Prevention: Life Expectancy by state**
The last available data was 2021.

In [ ]:
!pip install sodapy
import pandas as pd
from sodapy import Socrata

In [ ]:
cdc = Socrata("data.cdc.gov", None)
life = cdc.get("it4f-frdc", limit=2000)
life_df = pd.DataFrame.from_records(life)
life_df = life_df[life_df["sex"]=="Total"]
life_df["leb"] = life_df["leb"].astype(float)
life_df.rename(columns={'leb': 'life_expectancy'}, inplace=True)

### **1.5 Bureau of Labor Statistics: Labor Productivity**

In [ ]:
lab_pdt = pd.read_csv('https://raw.githubusercontent.com/JiaYue-Ong/Python-Final-Project/refs/heads/main/labor-productivity.csv')
lab_pdt.head()

In [30]:
lab_pdt_2023 = lab_pdt[lab_pdt['Area'].isin(state_names.values())]
lab_pdt_2023 = lab_pdt_2023[["Area","2023"]]
lab_pdt_2023.rename(columns={'Area': 'State', '2023': 'Labor_Productivity_2023'}, inplace=True)

## 2. Data Wrangling

Dependent variables:
1. GDP per capita
2. Life expectancy
3. Median household income
4. Education levels

Independent variables
1. Unemployment rate
2. Labour force participation rate
3. Labour Productivity Private non-farm
4. Population
5. Poverty rate
6. Net exports by state


In [ ]:
# All dataset
USA_acs_data
US_netexport_2023
bea_state_tbl
life_df
lab_pdt_2023

In [32]:
# Join ACS and netexport
df1 = USA_acs_data.merge(US_netexport_2023[['STATE_NAME', 'netexport']], left_on='NAME', right_on='STATE_NAME', how='left').drop(
    columns=["STATE_NAME"]
)

# Join bea_state_tbl
df2 = df1.merge(bea_state_tbl[['GeoName', 'REALGDP']], left_on='NAME', right_on='GeoName', how='left').drop(
    columns=["GeoName"]
)

# Join life_df
df3 = df2.merge(life_df[['area', 'life_expectancy']], left_on='NAME', right_on='area', how='left').drop(
    columns=["area"]
)

# Join lab_pdt_2023
df4 = df3.merge(lab_pdt_2023, left_on='NAME', right_on='State', how='left').drop(
    columns=["State"]
)

# Create Real GDP per capita
df4["REALGDPpercapita"] = df4["REALGDP"]/df4["TotalPop"]

In [33]:
# Get geographies of US States
import pygris
from pygris import states
from pygris.utils import shift_geometry

In [ ]:
us = states(cb = True, resolution = "20m", year=2023)
us_rescaled = shift_geometry(us)
us_rescaled.plot()

In [ ]:
# Join the data to geography
us_rescaled.head()

In [36]:
us_rescaled_final = us_rescaled.merge(
    df4,
    left_on=["GEOID"],
    right_on=["state"],
).drop(
    columns=["state"]
)

# Convert CRS
us_rescaled_final = us_rescaled_final.to_crs("EPSG:4326")

In [ ]:
us_rescaled_final.columns

## 3. Exploratory Plots

### 3.1 Correlation Matrix

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Create a list of all variables
variables = ['MedHHInc','TotalPop', 'TotalPop16', 'LabForTotal', 'Unemployed','PctBach', 'PovertyRate', 'UnemploymentRate', 'LabForParticipationRate', 'netexport', 'REALGDP', 'life_expectancy', 'Labor_Productivity_2023', 'REALGDPpercapita']

# Create a list of selected variables for later analysis
selected_variables = ['REALGDPpercapita','life_expectancy','MedHHInc','PctBach','UnemploymentRate','LabForParticipationRate', 'Labor_Productivity_2023', 'TotalPop', 'PovertyRate', 'netexport']

# Calculate the correlation matrix
corr_matrix = us_rescaled_final[variables].corr()

# Plot the correlation matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

### 3.2 Repeated Chart and Bubble Plot

In [40]:
import altair as alt

In [ ]:
# Setup the selection brush
brush = alt.selection_interval()

# Repeated chart
(
    alt.Chart(us_rescaled_final)
    .mark_circle()
    .encode(
        x=alt.X(alt.repeat("column"), type="quantitative", scale=alt.Scale(zero=False)),
        y=alt.Y(alt.repeat("row"), type="quantitative", scale=alt.Scale(zero=False)),
        color=alt.condition(
            brush, "NAME_x:N", alt.value("lightgray")
        ),  # conditional color
        tooltip=['NAME_x'] + variables
    )
    .properties(
        width=200,
        height=200,
    )
    .add_params(brush)
    .repeat(  # repeat variables across rows and columns
        row=variables,
        column=variables,
    )
)


In [ ]:
# Define dropdown bindings for both x and y axes
dropdown_x = alt.binding_select(
    options=['MedHHInc','TotalPop', 'TotalPop16', 'LabForTotal', 'Unemployed','PctBach', 'PovertyRate', 'UnemploymentRate', 'LabForParticipationRate', 'netexport', 'REALGDP', 'life_expectancy', 'Labor_Productivity_2023'],
    name='X-axis column '
)
dropdown_y = alt.binding_select(
    options=['MedHHInc','TotalPop', 'TotalPop16', 'LabForTotal', 'Unemployed','PctBach', 'PovertyRate', 'UnemploymentRate', 'LabForParticipationRate', 'netexport', 'REALGDP', 'life_expectancy', 'Labor_Productivity_2023'],
    name='Y-axis column '
)

# Create parameters for x and y axes
xcol_param = alt.param(
    value='MedHHInc',
    bind=dropdown_x
)
ycol_param = alt.param(
    value='MedHHInc',
    bind=dropdown_y
)

chart = alt.Chart(us_rescaled_final).mark_circle().encode(
    x=alt.X('x:Q', scale=alt.Scale(zero=False, domain='unaggregated')).title(''),
    y=alt.Y('y:Q', scale=alt.Scale(zero=False, domain='unaggregated')).title(''),
    size='TotalPop:Q',
    color='NAME_x:N',
    tooltip=['NAME_x'] + variables  # Concatenate NAME_x with the existing variables list
).transform_calculate(
    x=f'datum[{xcol_param.name}]',
    y=f'datum[{ycol_param.name}]'
).add_params(
    xcol_param,
    ycol_param,
).properties(width=800, height=800)

chart

In [ ]:
# Define dropdown bindings for both x and y axes
dropdown_x = alt.binding_select(
    options=['MedHHInc','TotalPop', 'TotalPop16', 'LabForTotal', 'Unemployed','PctBach', 'PovertyRate', 'UnemploymentRate', 'LabForParticipationRate', 'netexport', 'REALGDP', 'life_expectancy', 'Labor_Productivity_2023'],
    name='X-axis column '
)
dropdown_y = alt.binding_select(
    options=['MedHHInc','TotalPop', 'TotalPop16', 'LabForTotal', 'Unemployed','PctBach', 'PovertyRate', 'UnemploymentRate', 'LabForParticipationRate', 'netexport', 'REALGDP', 'life_expectancy', 'Labor_Productivity_2023'],
    name='Y-axis column '
)
dropdown_size = alt.binding_select(
    options=['MedHHInc','TotalPop', 'TotalPop16', 'LabForTotal', 'Unemployed','PctBach', 'PovertyRate', 'UnemploymentRate', 'LabForParticipationRate', 'netexport', 'REALGDP', 'life_expectancy', 'Labor_Productivity_2023'],
    name='Bubble Size '
)

# Create parameters for x and y axes
xcol_param = alt.param(
    value='MedHHInc',
    bind=dropdown_x
)
ycol_param = alt.param(
    value='MedHHInc',
    bind=dropdown_y
)
size_param = alt.param(
    value='MedHHInc',
    bind=dropdown_size
)

chart2 = alt.Chart(us_rescaled_final).mark_circle().encode(
    x=alt.X('x:Q', scale=alt.Scale(zero=False, domain='unaggregated')).title(''),
    y=alt.Y('y:Q', scale=alt.Scale(zero=False, domain='unaggregated')).title(''),
    size=alt.Size('size:Q', scale=alt.Scale(zero=False, domain='unaggregated')).title(''),
    color='NAME_x:N',
    tooltip=['NAME_x'] + variables  # Concatenate NAME_x with the existing variables list
).transform_calculate(
    x=f'datum[{xcol_param.name}]',
    y=f'datum[{ycol_param.name}]',
    size=f'datum[{size_param.name}]'
).add_params(
    xcol_param,
    ycol_param,
    size_param,
).properties(width=800, height=800)

chart2

### 3.3 Map

In [ ]:
pip install geopandas hvplot panel


In [ ]:
import geopandas as gpd
import hvplot.pandas
import panel as pn

In [46]:
# Convert from wide to long data
us_rescaled_final_long = pd.melt(us_rescaled_final,
                                 id_vars = ['STATEFP', 'STATENS', 'GEOIDFQ', 'GEOID', 'STUSPS', 'NAME_x', 'LSAD','ALAND', 'AWATER', 'geometry', 'NAME_y', 'GEO_ID'],
                                 value_vars=['MedHHInc', 'EducTotal', 'EducBelowHighSch', 'EducHighSch', 'EducAssoc', 'EducBach', 'TotalPop', 'TotalPop16', 'LabForTotal', 'Unemployed', 'PopPovertyDetermined', 'PovertyPop', 'PctBach', 'PovertyRate', 'UnemploymentRate', 'LabForParticipationRate', 'netexport', 'REALGDP', 'life_expectancy', 'Labor_Productivity_2023', 'REALGDPpercapita']
                                 )

In [ ]:
us_rescaled_final_long.hvplot(
    c="value",
    dynamic=False,
    width=1000,
    height=1000,
    geo=True,
    cmap="viridis",
    groupby="variable",
    )

# 4. Cluster Analysis

In [ ]:
!pip install kneed
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import StandardScaler

In [49]:
scaler = StandardScaler()

In [50]:
us_rescaled_final_scaled = scaler.fit_transform(us_rescaled_final[selected_variables])

In [ ]:
us_rescaled_final_scaled[:5]

In [ ]:
# Number of clusters to try out
n_clusters = list(range(2, 10))

# Run kmeans for each value of k
inertias = []
for k in n_clusters:
    
    # Initialize and run
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(us_rescaled_final_scaled)
    
    # Save the "inertia"
    inertias.append(kmeans.inertia_)

# Initialize the knee algorithm
kn = KneeLocator(n_clusters, inertias, curve='convex', direction='decreasing')

# Print out the knee 
print(kn.knee)

In [ ]:
kmeans = KMeans(n_clusters=5, n_init=10)

# Perform the fit
kmeans.fit(us_rescaled_final_scaled)

# Extract the labels
us_rescaled_final['label'] = kmeans.labels_

In [ ]:
# Average feature per cluster
us_rescaled_final.groupby('label', as_index=False).size()

In [ ]:
us_rescaled_final.groupby("label", as_index=False)[
    selected_variables
].mean().sort_values(by="label")

In [ ]:
# Map plot clusters
us_rescaled_final.hvplot(
    c="label",
    dynamic=False,
    width=1000,
    height=1000,
    geo=True,
    cmap="viridis",
    )

## 5. Predictive Modelling

In [61]:
# Models
from sklearn.ensemble import RandomForestRegressor

# Model selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Pipelines
from sklearn.pipeline import make_pipeline

# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [ ]:
selected_variables

In [159]:
usa_modelling = us_rescaled_final[selected_variables + ["geometry"]].dropna()

In [160]:
# Split the data 70/30
train_set, test_set = train_test_split(usa_modelling, test_size=0.3, random_state=42)

### 5.1 GDP per capita against variables

In [ ]:
# the target labels: log of sale price
y_GDPtrain = np.log(train_set["REALGDPpercapita"])
y_GDPtest = np.log(test_set["REALGDPpercapita"])

In [162]:
# The features
GDPfeature_cols = [
    'life_expectancy',
    'MedHHInc',
    'PctBach',
    'UnemploymentRate',
    'LabForParticipationRate',
    'Labor_Productivity_2023',
    'TotalPop',
    'PovertyRate',
    'netexport',
]
X_GDPtrain = train_set[GDPfeature_cols].values
X_GDPtest = test_set[GDPfeature_cols].values

In [ ]:
# Make a random forest pipeline
forest_pipeline = make_pipeline(
    StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=42)
)

# Run the 10-fold cross validation
GDPscores = cross_val_score(
    forest_pipeline,
    X_GDPtrain,
    y_GDPtrain,
    cv=10,
)

# Report
print("R^2 scores = ", GDPscores)
print("Scores mean = ", GDPscores.mean())
print("Score std dev = ", GDPscores.std())

In [ ]:
# Fit on the training data
forest_pipeline.fit(X_GDPtrain, y_GDPtrain)

# What's the test score?
forest_pipeline.score(X_GDPtest, y_GDPtest)

In [165]:
# Extract the regressor from the pipeline
forest_model = forest_pipeline["randomforestregressor"]

In [ ]:
# Create the data frame of importances
importanceGDP = pd.DataFrame(
    {"Feature": GDPfeature_cols, "Importance": forest_model.feature_importances_}
).sort_values("Importance")


importanceGDP.hvplot.barh(x="Feature", y="Importance")

### 5.2 Life Expectancy against variables

In [167]:
# the target labels: log of sale price
y_LEtrain = np.log(train_set["life_expectancy"])
y_LEtest = np.log(test_set["life_expectancy"])

In [168]:
# The features
LEfeature_cols = [
    'REALGDPpercapita',
    'MedHHInc',
    'PctBach',
    'UnemploymentRate',
    'LabForParticipationRate',
    'Labor_Productivity_2023',
    'TotalPop',
    'PovertyRate',
    'netexport',
]
X_LEtrain = train_set[LEfeature_cols].values
X_LEtest = test_set[LEfeature_cols].values

In [ ]:
# Make a random forest pipeline
LEforest_pipeline = make_pipeline(
    StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=42)
)

# Run the 10-fold cross validation
LEscores = cross_val_score(
    forest_pipeline,
    X_LEtrain,
    y_LEtrain,
    cv=10,
)

# Report
print("R^2 scores = ", LEscores)
print("Scores mean = ", LEscores.mean())
print("Score std dev = ", LEscores.std())

In [ ]:
# Fit on the training data
LEforest_pipeline.fit(X_LEtrain, y_LEtrain)

# What's the test score?
LEforest_pipeline.score(X_LEtest, y_LEtest)

In [171]:
# Extract the regressor from the pipeline
LEforest_model = LEforest_pipeline["randomforestregressor"]

In [ ]:
# Create the data frame of importances
importanceLE = pd.DataFrame(
    {"Feature": LEfeature_cols, "Importance": LEforest_model.feature_importances_}
).sort_values("Importance")


importanceLE.hvplot.barh(x="Feature", y="Importance")

### 5.3 Median Household Income against variables

In [173]:
# the target labels: log of sale price
y_HHINCtrain = np.log(train_set["MedHHInc"])
y_HHINCtest = np.log(test_set["MedHHInc"])

In [174]:
# The features
HHINCfeature_cols = [
    'REALGDPpercapita',
    'life_expectancy',
    'PctBach',
    'UnemploymentRate',
    'LabForParticipationRate',
    'Labor_Productivity_2023',
    'TotalPop',
    'PovertyRate',
    'netexport',
]
X_HHINCtrain = train_set[HHINCfeature_cols].values
X_HHINCtest = test_set[HHINCfeature_cols].values

In [ ]:
# Make a random forest pipeline
HHINCforest_pipeline = make_pipeline(
    StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=42)
)

# Run the 10-fold cross validation
HHINCscores = cross_val_score(
    forest_pipeline,
    X_HHINCtrain,
    y_HHINCtrain,
    cv=10,
)

# Report
print("R^2 scores = ", HHINCscores)
print("Scores mean = ", HHINCscores.mean())
print("Score std dev = ", HHINCscores.std())

In [ ]:
# Fit on the training data
HHINCforest_pipeline.fit(X_HHINCtrain, y_HHINCtrain)

# What's the test score?
HHINCforest_pipeline.score(X_HHINCtest, y_HHINCtest)

In [177]:
# Extract the regressor from the pipeline
HHINCforest_model = HHINCforest_pipeline["randomforestregressor"]

In [ ]:
# Create the data frame of importances
importanceHHINC = pd.DataFrame(
    {"Feature": HHINCfeature_cols, "Importance": HHINCforest_model.feature_importances_}
).sort_values("Importance")


importanceHHINC.hvplot.barh(x="Feature", y="Importance")

### 5.4 Education levels

In [179]:
# the target labels: log of sale price
y_PctBachtrain = np.log(train_set["PctBach"])
y_PctBachtest = np.log(test_set["PctBach"])

In [180]:
# The features
PctBachfeature_cols = [
    'REALGDPpercapita',
    'life_expectancy',
    'MedHHInc',
    'UnemploymentRate',
    'LabForParticipationRate',
    'Labor_Productivity_2023',
    'TotalPop',
    'PovertyRate',
    'netexport',
]
X_PctBachtrain = train_set[PctBachfeature_cols].values
X_PctBachtest = test_set[PctBachfeature_cols].values

In [ ]:
# Make a random forest pipeline
PctBachforest_pipeline = make_pipeline(
    StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=42)
)

# Run the 10-fold cross validation
PctBachscores = cross_val_score(
    forest_pipeline,
    X_PctBachtrain,
    y_PctBachtrain,
    cv=10,
)

# Report
print("R^2 scores = ", PctBachscores)
print("Scores mean = ", PctBachscores.mean())
print("Score std dev = ", PctBachscores.std())

In [ ]:
# Fit on the training data
PctBachforest_pipeline.fit(X_PctBachtrain, y_PctBachtrain)

# What's the test score?
PctBachforest_pipeline.score(X_PctBachtest, y_PctBachtest)

In [183]:
# Extract the regressor from the pipeline
PctBachforest_model = PctBachforest_pipeline["randomforestregressor"]

In [ ]:
# Create the data frame of importances
importancePctBach = pd.DataFrame(
    {"Feature": PctBachfeature_cols, "Importance": PctBachforest_model.feature_importances_}
).sort_values("Importance")


importancePctBach.hvplot.barh(x="Feature", y="Importance")